# [Build a Retrieval Augmented Generation (RAG) App](https://python.langchain.com/docs/tutorials/rag/)

テキスト データ ソースを使用して簡単な Q&A アプリケーションを構築する

## ■ RAGのコンポーネント

一般的な RAG アプリケーションには、次の 2 つの主要コンポーネントがあります。

- **インデックス作成**  
ソースからデータを取り込んでインデックスを作成するパイプライン。これは通常、オフラインで行われます。
- **検索と生成**  
実際の RAG チェーンは、実行時にユーザークエリを受け取り、インデックスから関連データを取得して、それをモデルに渡します。

### # インデックス生成の流れ


1. **ロード**  
まずデータをロードする必要があります。これは[Document Loaders](https://python.langchain.com/docs/concepts/document_loaders/)を使用して行われます。
1. **分割**  
[Text splitters](https://python.langchain.com/docs/concepts/text_splitters/)は、大きな `Documents` を小さなチャンクに分割します。  
これは、データのインデックス作成とモデルへの受け渡しの両方に役立ちます。  
大きなチャンクは検索が難しく、モデルの有限のコンテキストウィンドウに収まらないためです。  
1. **保存**  
分割したものを保存してインデックス化し、後で検索できるようにする場所が必要です。  
これは通常、[VectorStore](https://python.langchain.com/docs/concepts/vectorstores/)と[Embeddings](https://python.langchain.com/docs/concepts/embedding_models/)モデルを使用して行われます。  

<img src="../../docs/img/05_rag_app/01_indexing.png" width="700px">

### # 検索と生成の流れ


1. **検索**  
ユーザ入力が与えられると、[Retriever](https://python.langchain.com/docs/concepts/retrievers/)が関連するスプリットを取得します。
1. **生成**  
[ChatModel](https://python.langchain.com/docs/concepts/chat_models/) / [LLM](https://python.langchain.com/docs/concepts/text_llms/) は質問と検索されたデータの両方を含むプロンプトを使って回答を生成します。

<img src="../../docs/img/05_rag_app/02_retrieval.png" width="700px">

# プレビュー

In [2]:
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"

from typing import List
import bs4
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents.base import Document
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

#
# 1. ロード
#
bs4_strainer = bs4.SoupStrainer(class_=("it-MdContent"))
loader = WebBaseLoader(
    web_path=("https://qiita.com/ktamido/items/b4ea0c40c5e1957904f6",),
    bs_kwargs={
        "parse_only": bs4_strainer
    }
)
docs: List[Document] = loader.load()


#
# 2. 分割
#
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits: List[Document] = text_splitter.split_documents(docs)


#
# 3. 保存
#
vectorstore: Chroma = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())


#
# 4. 検索と生成
#

# モデル
llm = ChatOpenAI(model="gpt-4o")

# プロンプト
template = """
あなたは質問応答のアシスタントです。質問に答えるために、検索された文脈の以下の部分を使用してください。答えがわからない場合は、わからないと答えましょう。回答は3文以内で簡潔に。

Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate([
    HumanMessagePromptTemplate.from_template(template),
])

# Retriever
retriever = vectorstore.as_retriever()
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)


# 5. 実行
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain.invoke("リポジトリにはどのような役割がありますか？")

'リポジトリは、ユーザー情報の保存、取得、更新、削除を担当する役割を持っています。具体的には、データを永続化するための手段として使用されます。たとえば、`UserRepository`はユーザー情報をJSONファイルに保存し、必要に応じてデータを取得、更新、削除します。'

# インデックス: ロード

まず、[DocumentLoaders](https://python.langchain.com/docs/concepts/document_loaders/)を利用してブログ記事のコンテンツをロードします。  
[DocumentLoaders](https://python.langchain.com/docs/concepts/document_loaders/)は、ソースからデータを読み込み、 `Document` のリストを返します。`Document` は `page_content(str)` と `metadata(dict)` を要素に持ちます。

今回はブログの記事を読み込むので、[WebBaseLoader](https://python.langchain.com/docs/integrations/document_loaders/web_base/)を使用します。  
これは `urllib` を利用してWebからHTMLをロードし、 `BeautifulSoup` を使用してテキストにパースします。  
`bs_kwargs` を利用することで、 `BeautifulSourp` にパラメータを渡すことができます。 ([BeautifulSoupのドキュメント](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup))  
今回の場合、 `post-content` `post-title` `post-header` クラスを持つHTMLタグのみ取得し、それ以外は削除します。  

## 参考

- [各データソースにおけるDocumentLoaderの使い方](https://python.langchain.com/docs/how_to/#document-loaders)
- [DocumentLoaders ドキュメント](https://python.langchain.com/docs/integrations/document_loaders/)
  - [BaseLoader (DocumentLoadersのインターフェース)](https://python.langchain.com/api_reference/core/document_loaders/langchain_core.document_loaders.base.BaseLoader.html)





In [ ]:
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"

from typing import List
from langchain_core.documents.base import Document

import bs4
from langchain_community.document_loaders import WebBaseLoader

# post-title, post-content, post-headerのみ取得
bs4_strainer = bs4.SoupStrainer(class_=["post-title", "post-content", "post-header"])

# ロード
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer}  # BeautifulSoupの引数
)
docs: List[Document] = loader.load()

print(f"docs length: {len(docs)}")
print(f"content length: {len(docs[0].page_content)}")
print(f"metadata: {docs[0].metadata}")


docs length: 1
content length: 43131
metadata: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}


In [2]:
# 取得したデータを表示
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


# インデックス: 分割

ドキュメントを1000文字のチャンクに分割し、チャンク間に200文字のオーバーラップ(文とそれに関連する重要な文脈が分離してしまう可能性を軽減する)を設けます。  
[RecursiveCharacterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter/) を使用し、各チャンクが適切なサイズになるまで、改行などの一般的な区切り文字を使用してドキュメントを再帰的に分割します。これは、一般的なテキストを使う場合の推奨テキストスプリッタです。

`add_start_index=True` を設定し、分割されたDocumentが元の文章の何文字目から始まるのかを示すメタデータ属性 `start_index` を保持するようにします。

In [3]:
from typing import List
from langchain_core.documents.base import Document

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
all_splits: List[Document] = text_splitter.split_documents(docs)

print(f"docs length: {len(all_splits)}")
print(f"content length (index=9): {len(all_splits[9].page_content)}")
print(f"metadata (index=9): {all_splits[9].metadata}")

docs length: 66
content length (index=9): 960
metadata (index=9): {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 6095}


# インデックス: 保存

[Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma/)ベクトルストアと[OpenAIEmbeddings](https://python.langchain.com/docs/integrations/text_embedding/openai/)モデルを使って、ドキュメント分割をベクトル化し保存します。

## 参考

- Embeddings
  - [Embeddingsの使い方](https://python.langchain.com/docs/how_to/embed_text/)
  - [Embeddingモデル一覧](https://python.langchain.com/docs/integrations/text_embedding/)
- VectorStore
  - [VectorStoreの使い方](https://python.langchain.com/docs/how_to/vectorstores/)
  - [VectorStore一覧](https://python.langchain.com/docs/integrations/vectorstores/)
  - [インターフェース](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html)

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# 検索と生成: 検索

最も一般的なRetrieverは [VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html) で、ベクトルストアの類似検索機能を利用して検索を行います。  
どの `VectorStore` も `VectorStore.as_retriever()` メソッドで簡単に `Retriever` オブジェクトを取得することができます。

## 参考

- [vectorestoreをretrieverとして使うには？](https://python.langchain.com/docs/how_to/vectorstore_retriever/)
- [Retrieverの使い方一覧](https://python.langchain.com/docs/how_to/#retrievers)
- [Retriever一覧](https://python.langchain.com/docs/integrations/retrievers/)
- [BaseRetrieverインターフェース](https://python.langchain.com/api_reference/core/retrievers/langchain_core.retrievers.BaseRetriever.html)

In [9]:
from typing import List
from langchain_core.documents.base import Document
from langchain_core.vectorstores.base import VectorStoreRetriever

retriever: VectorStoreRetriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retrieved_docs: List[Document] = retriever.invoke("What are the approaches to Task Decomposition?")
print(f"retrieved docs: {len(retrieved_docs)}")
print(retrieved_docs[0].page_content)


retrieved docs: 5
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


# 検索と生成: 生成


In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

ここでは、LangChainの[prompt hub](https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=1f22096d-1ee0-451f-9572-c1bdbc2ef487)にチェックインされているRAG用プロンプトを使用する。

In [ ]:
from langchain import hub

# プロンプトのテンプレートを取得
# https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=1f22096d-1ee0-451f-9572-c1bdbc2ef487
prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

print(example_messages)
print(example_messages[0].content)


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [ ]:
from typing import List
from langchain_core.documents.base import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# retrieverから取得したDocumentを文字列に成形
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task decomposition is the process of breaking down complex tasks into smaller, more manageable steps. This can be achieved using techniques like Chain of Thought (CoT) and Tree of Thoughts, which encourage systematic reasoning and exploration of multiple possibilities. By doing so, it enhances understanding and execution efficiency for both human and AI agents.

## 自前のプロンプトを使う

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# モデル
llm = ChatOpenAI(model="gpt-4o-mini")

# プロンプト
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = ChatPromptTemplate([
    HumanMessagePromptTemplate.from_template(template),
])

# retrieverから取得したDocumentを文字列に成形
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")


"Task decomposition is the process of breaking down a complicated task into smaller, manageable steps to facilitate planning and execution. Techniques like Chain of Thought (CoT) and Tree of Thoughts enhance this process by enabling models to reason step-by-step and explore multiple possibilities. This approach helps improve performance on complex tasks and clarifies the model's reasoning. Thanks for asking!"